# Train model

In [1]:
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
import json

In [3]:
# Your service key JSON file relative to this notebook
aic_service_key_path = '../files/aic_service_key.json'

# Loads the service key file
with open(aic_service_key_path) as ask:
    aic_service_key = json.load(ask)

# Creating an AI API client instance
ai_api_client = AIAPIV2Client(
    base_url = aic_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aic_service_key["url"] + "/oauth/token",
    client_id = aic_service_key['clientid'],
    client_secret = aic_service_key['clientsecret']
)

# Connect Docker to SAP AI Core

### Already done in defect detection exercise

# Create and upload the docker images

Start docker on your machine and execute the following commands in a terminal:

docker login docker.io -u YOUR_DOCKER_REPO

docker build code/train -t YOUR_DOCKER_REPO/sound-train:0.0.1 

docker push docker.io/YOUR_DOCKER_REPO/sound-train:0.0.1

NB: you should be connected to SAP VPN to download sap-ai-tracking module

# Create Resource Group and Connect AWS S3 to SAP AI Core

In [31]:
# Loads your service key
s3_service_key_path = '../files/s3_service_key.json'

# Loads the service key file
with open(s3_service_key_path) as s3sk:
    s3_service_key = json.load(s3sk)

default_secret = {
    "name": "default", # Name of the connection
    "type": "S3",
    "endpoint": s3_service_key["host"],
    "bucket": s3_service_key["bucket"],
    "pathPrefix": "sound",
    "region": s3_service_key["region"],
    "data": {
        "AWS_ACCESS_KEY_ID": s3_service_key["access_key_id"],
        "AWS_SECRET_ACCESS_KEY": s3_service_key["secret_access_key"]
    }
}

# Call the api
ai_api_client.rest_client.post(
    path="/admin/objectStoreSecrets",
    body = default_secret, # defined above
    resource_group = "sound"
)

{'message': 'Secret exists'}

In [32]:
ai_api_client.rest_client.get(
    path="/admin/objectStoreSecrets",
    resource_group = "sound"
)

{'count': 2,
 'resources': [{'metadata': {'serving.kubeflow.org/s3-endpoint': 's3.amazonaws.com',
    'serving.kubeflow.org/s3-region': 'eu-central-1',
    'serving.kubeflow.org/s3-usehttps': '',
    'serving.kubeflow.org/s3-verifyssl': '',
    'storage.ai.sap.com/bucket': 'books-dataset',
    'storage.ai.sap.com/endpoint': 's3.amazonaws.com',
    'storage.ai.sap.com/path_prefix': 'sound',
    'storage.ai.sap.com/region': 'eu-central-1',
    'storage.ai.sap.com/type': 'S3'},
   'name': 'default'},
  {'metadata': {'serving.kubeflow.org/s3-endpoint': 's3.amazonaws.com',
    'serving.kubeflow.org/s3-region': 'eu-central-1',
    'serving.kubeflow.org/s3-usehttps': '',
    'serving.kubeflow.org/s3-verifyssl': '',
    'storage.ai.sap.com/bucket': 'books-dataset',
    'storage.ai.sap.com/endpoint': 's3.amazonaws.com',
    'storage.ai.sap.com/path_prefix': 'sound',
    'storage.ai.sap.com/region': 'eu-central-1',
    'storage.ai.sap.com/type': 'S3'},
   'name': 'sound-data'}]}

# Train Execution of ML Model

In [33]:
import sys, os
import json
import requests
import base64
import time
import yaml
from IPython.display import clear_output
from pprint import pprint

from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.status import Status
from ai_api_client_sdk.models.target_status import TargetStatus
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding

In [34]:
resource_group = "sound"  # Must be created before

aic_service_key = '../files/aic_service_key.json' # ENSURE YOU HAVE THE FILE PLACED CORRECTLY
with open(aic_service_key) as ask:
    aic_s_k = json.load(ask)

# NO CHANGES REQUIRED BELOW
#
ai_api_v2_client = AIAPIV2Client(
    base_url=aic_s_k["serviceurls"]["AI_API_URL"] + "/v2/lm",
    auth_url=aic_s_k["url"] + "/oauth/token",
    client_id=aic_s_k['clientid'],
    client_secret=aic_s_k['clientsecret'],
    resource_group=resource_group)

Check the available scenarios

In [35]:
scenario_resp = ai_api_v2_client.scenario.query(resource_group)

print("Scenarios")
print("---" * 20)
for idx, scenario in enumerate(scenario_resp.resources):
    print(f"Scenario ID {idx + 1} -> [{scenario.id}]")

Scenarios
------------------------------------------------------------
Scenario ID 1 -> [text-clf-tutorial]
Scenario ID 2 -> [image-clf-tutorial]
Scenario ID 3 -> [sound-scenario]


Register training data as artifact

In [36]:
# NO CHANGES REQUIRED BELOW
#
# Load training_workflow.yaml
training_workflow_file = '../workflows_sound/training_workflow.yaml'
with open(training_workflow_file) as twf:
    training_workflow = yaml.safe_load(twf)
#
# Load scenario id from train_workflow.yaml
scenario_id = training_workflow['metadata']['labels']['scenarios.ai.sap.com/id']
#
# Set the artifact configuration
artifact = {
        "name": "sound-data", # Modifiable Name
        "kind": Artifact.Kind.DATASET,
        "url": "ai://default/data",
        "description": "Cutting machine sound clips for defect detection",
        "scenario_id": scenario_id
    }
# Store the artifact response to retrieve the id for the training configuration
artifact_resp = ai_api_v2_client.artifact.create(**artifact)
print(f"Artifacts registered for {scenario_id} scenario!")
pprint(vars(artifact_resp))
#
# Checks if the message contains expected string
assert artifact_resp.message == 'Artifact acknowledged'

Artifacts registered for sound-scenario scenario!
{'id': 'df27e7db-75ec-4062-83d4-68fd1a6c1613',
 'message': 'Artifact acknowledged',
 'url': 'ai://default/data'}


Create training configuration

In [37]:
input_artifact_name = training_workflow['spec']['templates'][0]['inputs']['artifacts'][0]['name']
executable_name = training_workflow['metadata']['name']

artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": artifact_resp.id
}

train_configuration = {
    "name": "sound-training-configuration",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

# store the configuration response to access the id to create an execution
train_config_resp = ai_api_v2_client.configuration.create(**train_configuration)
pprint(vars(train_config_resp))

assert train_config_resp.message == 'Configuration created'

print("Configuration created for running the training")

{'id': '93972c46-123a-4b4a-9d1e-8b929929f475',
 'message': 'Configuration created'}
Configuration created for running the training


Create training execution

In [38]:
execution_resp = ai_api_v2_client.execution.create(train_config_resp.id)
pprint(vars(execution_resp))

{'id': 'e23e9b4c545cb06f',
 'message': 'Execution scheduled',
 'status': <Status.UNKNOWN: 'UNKNOWN'>}


Observe training status

In [39]:
status = None
while status != Status.COMPLETED and status != Status.DEAD:
    # Sleep for 5 secs to avoid overwhelming the API with requests
    time.sleep(5)
    # Clear outputs to reduce clutter
    clear_output(wait=True)

    execution = ai_api_v2_client.execution.get(execution_resp.id)
    status = execution.status
    print('...... execution status ......', flush=True)
    print(f"Training status: {execution.status}")
    print(f"Training status details: {execution.status_details}")


if execution.status == Status.COMPLETED:
    print(f"Training complete for execution [{execution_resp.id}]!")
    output_artifact = execution.output_artifacts[0]
    output = {
        "id": output_artifact.id,
        "name": output_artifact.name,
        "url": output_artifact.url
    }
    with open('training_output.json', 'w') as fp:
        json.dump(output, fp)

...... execution status ......
Training status: Status.DEAD
Training status details: {'details': [{'container_name': 'main', 'exit_code': 1, 'last_log_messages': '    *** = ***(py_func)  File ***/***", line 351, in __init__    "for file %r" % (qualname, source_path))RuntimeError: cannot cache function \'__shear_dense\': no locator available for file ***/***\'...', 'message': '', 'pod_name': 'e23e9b4c545cb06f', 'ready': False}, {'container_name': 'wait', 'exit_code': 0, 'last_log_messages': 'd65633d 082797e7b7f7e722c4d45ad4586eee998ba4ac5a873207174a0723ffdcbbe14f] killed successfully"time="2022-03-01T23:02:***" level=info msg="Alloc=10570 TotalAlloc=19673 Sys=73297 NumGC=5 Goroutines=8"...', 'message': '', 'pod_name': 'e23e9b4c545cb06f', 'ready': False}], 'workflow_info': [{'exit_code': '1', 'id': 'e23e9b4c545cb06f', 'message': 'Error (exit code 1)', 'name': 'e23e9b4c545cb06f', 'outputs': '', 'phase': 'Failed', 'type': 'Pod'}]}


# Deploy ML Model

In [ ]:
serving_workflow_file = 'workflows/serving_workflow_tutorial.yaml'
with open(serving_workflow_file) as swf:
    serving_workflow = yaml.safe_load(swf)

scenario_id = serving_workflow['metadata']['labels']['scenarios.ai.sap.com/id']
input_artifact_name = serving_workflow['spec']['inputs']['artifacts'][0]['name']
executable_name = serving_workflow['metadata']['name']

training_output = 'training_output.json'
with open(training_output) as to:
    serving_input = json.load(to)

In [ ]:
artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": serving_input["id"]
}

serve_configuration = {
    "name": "dev-tutorial-serving-configuration",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

serve_config_resp = ai_api_v2_client.configuration.create(**serve_configuration)

assert serve_config_resp.message == 'Configuration created'

pprint(vars(serve_config_resp))
print("configuration for serving the model created")

In [ ]:
deployment_resp = ai_api_v2_client.deployment.create(serve_config_resp.id)
pprint(vars(deployment_resp))

In [ ]:
# Poll deployment status
status = None
while status != Status.RUNNING and status != Status.DEAD:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print('...... deployment status ......', flush=True)
    print(deployment.status)
    pprint(deployment.status_details)

    if deployment.status == Status.RUNNING:
        print(f"Deployment with {deployment_resp.id} complete!")

# Allow some time for deployment URL to get ready
time.sleep(10)

# Using deployed ML model

In [ ]:
# Preparing the input for inference
# prediciton: compliment
test_input = {
    "text": "I loved this food, it was very good"
}

endpoint = f"{deployment.deployment_url}/v1/models/textmodel:predict"
headers = {"Authorization": ai_api_v2_client.rest_client.get_token(),
           'ai-resource-group': resource_group,
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print('Inference result:', response.json())
pprint(vars(response))

In [ ]:
# Preparing the input for inference
# prediction: complaint
test_input = {
    "text": "I hated this food, it was very bad"
}
endpoint = f"{deployment.deployment_url}/v1/models/textmodel:predict"
headers = {"Authorization": ai_api_v2_client.rest_client.get_token(),
           'ai-resource-group': resource_group,
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print('Inference result:', response.json())
pprint(vars(response))

# Stop deployed model

In [ ]:
delete_resp = ai_api_v2_client.deployment.modify(deployment_resp.id,
                                                 target_status=TargetStatus.STOPPED)
status = None
while status != Status.STOPPED:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print('...... killing deployment ......', flush=True)
    print(f"Deployment status: {deployment.status}")